In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D,Activation
from keras import backend as K
import tensorflow as tf
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import img_to_array

# 导入必要的软件包
import datetime
import imutils
import time
import cv2
import threading

Using TensorFlow backend.


In [2]:
input_shape = (150, 150, 3)

# 模型创建模型
def create_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(7)) #4分类
    model.add(Activation('softmax')) #采用Softmax

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model

In [3]:
number = ['BLACK_CHE','BLACK_JIANG','BLACK_MA','BLACK_PAO','BLACK_SHI','BLACK_XIANG','BLACK_ZU']
#恢复模型
model = create_model()
model.load_weights('weights.h5')

In [4]:
def predict(image):
    result = model.predict(a)
    result = np.argmax(result[0])
    return result

def image_dealwith(img):
    #图像灰度处理
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)

    gradX = cv2.Sobel(gray, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)
    gradY = cv2.Sobel(gray, ddepth=cv2.CV_32F, dx=0, dy=1, ksize=-1)
    # subtract the y-gradient from the x-gradient
    gradient = cv2.subtract(gradX, gradY)
    gradient = cv2.convertScaleAbs(gradient)

    # blur and threshold the image
    blurred = cv2.blur(gradient, (9, 9))
    (_, thresh) = cv2.threshold(blurred, 140, 120, cv2.THRESH_BINARY)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 25))
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    # perform a series of erosions and dilations
    closed = cv2.erode(closed, None, iterations=4)
    closed = cv2.dilate(closed, None, iterations=4)
    return closed

In [ ]:
# camera = cv2.VideoCapture(0)
camera = cv2.VideoCapture(1)
time.sleep(0.25)

firstFrame = None


# 遍历视频的每一帧
while True:
    # 获取当前帧并初始化occupied/unoccupied文本
    (grabbed, frame) = camera.read()
 
    # 如果不能抓取到一帧，说明我们到了视频的结尾
    if not grabbed:
        break
 
    # 调整该帧的大小，转换为灰阶图像并且对其进行高斯模
    closed = image_dealwith(frame)
    (_,cnts, _) = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if cnts is None:
        cv2.putText(frame, "Room Status: None", (10, 20),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        cv2.imshow("Security Feed", frame)
        key = cv2.waitKey(1)
    else:
        for c in cnts:
            rect = cv2.contourArea(c)
            #可以通过if语句筛选掉小框
            (x, y, w, h) = cv2.boundingRect(c)
            crop_imgae = frame[y:y+h,x:x+w]#分割图像进行识别
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

            Grayimg = cv2.cvtColor(crop_imgae, cv2.COLOR_BGR2RGB)
#             fff = cv2.resize(Grayimg, (500, 500))

            image = cv2.resize(Grayimg, (150, 150))
            image = image.astype("float") / 255.0
            image = img_to_array(image)
            image = np.expand_dims(image, axis=0)

            predict = model.predict(image, verbose=0)
            index = np.argmax(predict)

            cv2.putText(fff, "Room Status: {}".format(number[index]), (10, 20),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        #     cv2.putText(frame, "Room Status: {}".format(result), (10, 20),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

            #显示当前帧并记录用户是否按下按键
            cv2.imshow("Security Feed", frame)
            key = cv2.waitKey(1)

        # 如果q键被按下，跳出循环
        if key == ord("q"):
            break

 # 清理摄像机资源并关闭打开的窗口
camera.release()
cv2.destroyAllWindows()